## Data Fetching using NVD URL

In [ ]:
import pandas as pd
import requests
import zipfile
import os
from pandas import json_normalize

# Function to download and extract CVE data for a specific year
def download_cve_data(year):
    CVE_FEED = f'nvdcve-1.1-{year}.json'
    CVE_URL = f'https://nvd.nist.gov/feeds/json/cve/1.1/{CVE_FEED}.zip'

    # Download the CVE feed
    response = requests.get(CVE_URL)

    # Check if the download was successful
    if response.status_code != 200:
        print(f"Failed to download data for year {year}. Status code: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame on failure

    # Save the ZIP file locally
    zip_file_path = f'{CVE_FEED}.zip'
    with open(zip_file_path, 'wb') as f:
        f.write(response.content)

    # Validate that the downloaded file is indeed a ZIP file
    try:
        # Unzip the file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall()  # Extract to current directory
    except zipfile.BadZipFile:
        print(f"Downloaded file for year {year} is not a valid ZIP file.")
        os.remove(zip_file_path)  # Clean up
        return pd.DataFrame()  # Return an empty DataFrame on failure

    # Load the JSON data
    cvejson = pd.read_json(CVE_FEED)

    # Normalize the CVE items
    cve_all_raw = json_normalize(cvejson['CVE_Items'])

    # Function to extract relevant fields
    def extract_cve_data(row):
        vendor = row['cve.CVE_data_meta.ASSIGNER'].split('@')[-1].split('.')[0] if '@' in row['cve.CVE_data_meta.ASSIGNER'] else None

        return {
            'cveID': row['cve.CVE_data_meta.ID'],
            'pubDate': row['publishedDate'][:10],  # Extract date only
            'lastModDate': row['lastModifiedDate'][:10],  # Extract date only
            'source': row['cve.data_format'],  # CVE Data Format
            'description': row['cve.description.description_data'][0]['value'],  # CVE Description
            'cvssVector': row['impact.baseMetricV2.cvssV2.vectorString'],  # CVSS Vector String
            'accessVector': row['impact.baseMetricV2.cvssV2.accessVector'],  # Access Vector
            'accessComplexity': row['impact.baseMetricV2.cvssV2.accessComplexity'],  # Access Complexity
            'confidentialityImpact': row['impact.baseMetricV2.cvssV2.confidentialityImpact'],  # Confidentiality Impact
            'integrityImpact': row['impact.baseMetricV2.cvssV2.integrityImpact'],  # Integrity Impact
            'availabilityImpact': row['impact.baseMetricV2.cvssV2.availabilityImpact'],  # Availability Impact
            'baseScore': row['impact.baseMetricV2.cvssV2.baseScore'],  # Base Score
            'severity': row['impact.baseMetricV2.severity'],  # Severity
            'exploitabilityScore': row['impact.baseMetricV2.exploitabilityScore'],  # Exploitability Score
            'impactScore': row['impact.baseMetricV2.impactScore'],  # Impact Score
            'refURLs': [ref['url'] for ref in row['cve.references.reference_data']],  # References URLs
            'vendor': vendor  # Add vendor column
        }

    # Apply the extraction to all rows in the DataFrame
    extracted_data_list = cve_all_raw.apply(extract_cve_data, axis=1)

    # Convert to DataFrame
    extracted_df = pd.DataFrame(extracted_data_list.tolist())

    # Cleanup: Remove downloaded files
    os.remove(zip_file_path)
    os.remove(CVE_FEED)

    return extracted_df

# CVSS score calculation function
def calculate_cvss_base_score(av, ac, pr, ui):
    # Define scores for each metric option
    av_scores = {'N': 0.85, 'A': 0.62, 'L': 0.55, 'P': 0.2}
    ac_scores = {'L': 0.77, 'H': 0.44}
    pr_scores = {'N': 0.85, 'L': 0.62, 'H': 0.27}  # Assuming scope is unchanged
    ui_scores = {'N': 0.85, 'R': 0.62}

    # Retrieve scores from the metric values
    av_score = av_scores.get(av, 0)
    ac_score = ac_scores.get(ac, 0)
    pr_score = pr_scores.get(pr, 0)
    ui_score = ui_scores.get(ui, 0)

    # Calculate the CVSS Base Score
    impact = 1 - ((1 - av_score) * (1 - ac_score) * (1 - pr_score) * (1 - ui_score))
    exploitability = 8.22 * av_score * ac_score * pr_score * ui_score
    base_score = (impact + exploitability) * 10 / 2

    return round(base_score, 1)

# Function to extract values from cvssVector and calculate CVSS score
def extract_cvss_score(cvss_vector):
    if pd.isna(cvss_vector):  # Check for NaN
        return None  # or return 0, or some placeholder value

    # Split the vector string to extract values
    vector_parts = cvss_vector.split('/')

    # Create a dictionary from the parts
    metrics = {}
    for part in vector_parts:
        key, value = part.split(':')
        metrics[key] = value

    # Extract values from the metrics dictionary
    av = metrics.get('AV', None)
    ac = metrics.get('AC', None)
    pr = metrics.get('Au', None)  # 'Au' for Authentication
    ui = metrics.get('C', None)   # 'C' for Confidentiality

    # Map Confidentiality (C) to the appropriate parameter for scoring
    if metrics.get('C') == 'N':
        ui = 'N'  # No user interaction if Confidentiality is 'N'
    elif metrics.get('C') == 'P':
        ui = 'R'  # Required if Confidentiality is 'P'

    # Calculate CVSS score
    return calculate_cvss_base_score(av, ac, pr, ui)

# Loop through the years 2019 to 2023 and combine the data
df = pd.DataFrame()
for year in range(1999, 2024):  # Updated to include 2024
    yearly_data = download_cve_data(year)
    df = pd.concat([df, yearly_data], ignore_index=True)

# Add cvss_score column to the DataFrame
df['cvss_score'] = df['cvssVector'].apply(extract_cvss_score)

# Save the combined DataFrame to a CSV file
df.to_csv('cve_data.csv', index=False)  # index=False to exclude row numbers

# Save the DataFrame to a JSON file
df.to_json('cve_data.json', orient='records', lines=True)

print("Data saved to 'cve_data.csv' and 'cve_data.json'")
print(df.head())  # Show the first few entries

Failed to download data for year 1999. Status code: 404
Failed to download data for year 2000. Status code: 404
Failed to download data for year 2001. Status code: 404
Data saved to 'cve_data.csv' and 'cve_data.json'
           cveID     pubDate lastModDate source  \
0  CVE-1999-0001  1999-12-30  2010-12-16  MITRE   
1  CVE-1999-0002  1998-10-12  2009-01-26  MITRE   
2  CVE-1999-0003  1998-04-01  2018-10-30  MITRE   
3  CVE-1999-0004  1997-12-16  2018-10-12  MITRE   
4  CVE-1999-0005  1998-07-20  2008-09-09  MITRE   

                                         description  \
0  ip_input.c in BSD-derived TCP/IP implementatio...   
1  Buffer overflow in NFS mountd gives root acces...   
2  Execute commands as root via buffer overflow i...   
3  MIME buffer overflow in email clients, e.g. So...   
4  Arbitrary command execution via IMAP buffer ov...   

                   cvssVector accessVector accessComplexity  \
0  AV:N/AC:L/Au:N/C:N/I:N/A:P      NETWORK              LOW   
1  AV:N/AC:L/

## Initial Data Cleaning

In [ ]:
import pandas as pd
import numpy as np

# Load data
data = pd.read_csv('cve_data.csv')

# Drop duplicate entries based on CVE ID
data.drop_duplicates(subset="cveID", inplace=True)

# Handle missing values
# Filling empty fields with default text or zeros as appropriate
data.fillna({
    'description': 'No description provided',
    'cvssVector': 'N/A',
    'cvss_score': 0.0,
    'severity': 'Unknown'
}, inplace=True)

# Standardize date formats while keeping them as strings
data['pubDate'] = pd.to_datetime(data['pubDate'], errors='coerce').dt.strftime('%Y-%m-%d')
data['lastModDate'] = pd.to_datetime(data['lastModDate'], errors='coerce').dt.strftime('%Y-%m-%d')

# Save the cleaned data to a JSON file
data.to_json('cve_data_cleaned.json', orient='records', lines=True)

print("Data cleaning complete. Cleaned data saved to 'cve_data_cleaned.json'")
print(data.head())  # Display the first few entries of the cleaned data

<ipython-input-6-d598211f8e18>:5: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('cve_data.csv')


Data cleaning complete. Cleaned data saved to 'cve_data_cleaned.json'
           cveID     pubDate lastModDate source  \
0  CVE-1999-0001  1999-12-30  2010-12-16  MITRE   
1  CVE-1999-0002  1998-10-12  2009-01-26  MITRE   
2  CVE-1999-0003  1998-04-01  2018-10-30  MITRE   
3  CVE-1999-0004  1997-12-16  2018-10-12  MITRE   
4  CVE-1999-0005  1998-07-20  2008-09-09  MITRE   

                                         description  \
0  ip_input.c in BSD-derived TCP/IP implementatio...   
1  Buffer overflow in NFS mountd gives root acces...   
2  Execute commands as root via buffer overflow i...   
3  MIME buffer overflow in email clients, e.g. So...   
4  Arbitrary command execution via IMAP buffer ov...   

                   cvssVector accessVector accessComplexity  \
0  AV:N/AC:L/Au:N/C:N/I:N/A:P      NETWORK              LOW   
1  AV:N/AC:L/Au:N/C:C/I:C/A:C      NETWORK              LOW   
2  AV:N/AC:L/Au:N/C:C/I:C/A:C      NETWORK              LOW   
3  AV:N/AC:L/Au:N/C:N/I:N/A:P     

**Correct code**

In [9]:
import pandas as pd
import numpy as np
import json

# Load data
data = pd.read_csv('cve_data.csv')

# Drop duplicate entries based on CVE ID
data.drop_duplicates(subset="cveID", inplace=True)

# Handle missing values
data.fillna({
    'description': 'No description provided',
    'cvssVector': 'N/A',
    'cvss_score': 0.0,
    'severity': 'Unknown'
}, inplace=True)

# Standardize date formats while keeping them as strings
data['pubDate'] = pd.to_datetime(data['pubDate'], errors='coerce').dt.strftime('%Y-%m-%d')
data['lastModDate'] = pd.to_datetime(data['lastModDate'], errors='coerce').dt.strftime('%Y-%m-%d')

# Prepare data for saving in the desired JSON format
json_data = []
for _, row in data.iterrows():
    json_entry = {
        "id": row["cveID"],
        "description": row["description"],
        "cvssScore": str(row["cvss_score"]),  # Convert CVSS score to string
        "publishedDate": row["pubDate"],
        "affectedProduct": row["vendor"],  # Assuming 'vendor' contains affected product info
        "authenticationRequired": "No",  # Update based on your actual data if needed
        "accessComplexity": row["accessComplexity"],
        "confidentialityImpact": row["confidentialityImpact"],
        "integrityImpact": row["integrityImpact"],
        "availabilityImpact": row["availabilityImpact"],
        "references": row["refURLs"].split(',') if isinstance(row["refURLs"], str) else []  # Check for string type
    }
    json_data.append(json_entry)

# Save the prepared data to a JSON file with the specified name
with open('cve_cleaned_data_1.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

print("Data cleaning complete. Cleaned data saved to 'cve_cleaned_data_1.json'")
print(data.head())  # Display the first few entries of the cleaned data


<ipython-input-9-3ba1789dc6c8>:6: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('cve_data.csv')


Data cleaning complete. Cleaned data saved to 'cve_cleaned_data_1.json'
           cveID     pubDate lastModDate source  \
0  CVE-1999-0001  1999-12-30  2010-12-16  MITRE   
1  CVE-1999-0002  1998-10-12  2009-01-26  MITRE   
2  CVE-1999-0003  1998-04-01  2018-10-30  MITRE   
3  CVE-1999-0004  1997-12-16  2018-10-12  MITRE   
4  CVE-1999-0005  1998-07-20  2008-09-09  MITRE   

                                         description  \
0  ip_input.c in BSD-derived TCP/IP implementatio...   
1  Buffer overflow in NFS mountd gives root acces...   
2  Execute commands as root via buffer overflow i...   
3  MIME buffer overflow in email clients, e.g. So...   
4  Arbitrary command execution via IMAP buffer ov...   

                   cvssVector accessVector accessComplexity  \
0  AV:N/AC:L/Au:N/C:N/I:N/A:P      NETWORK              LOW   
1  AV:N/AC:L/Au:N/C:C/I:C/A:C      NETWORK              LOW   
2  AV:N/AC:L/Au:N/C:C/I:C/A:C      NETWORK              LOW   
3  AV:N/AC:L/Au:N/C:N/I:N/A:P   